In [1]:
import os
import sys
import math
from pathlib import Path
import argparse
import logging
import mimetypes

In [77]:
'''
Assume a path structure:

main_folder
|___ data(folder)
|     |__project_data_folder1
                    |__ dati_cebra.jl (file dati)
      |__project_data_folder2
                    |__ dati_mirco.mat (file dati mat file, jle file etc)
|___ project_root(folder)
            |__d_cod_mon_Mirco.py
            |__some_functions.py
            |___EEG_ANN_pipeline(folder)
                    |__data (folder)
                    |__helpers (folder)
                    |__etc....
            |___output directory (folder)
            

'''

'\nAssume a path structure:\n\nmain_folder\n|___ data(folder)\n|     |__project_data_folder1\n                    |__ dati_cebra.jl (file dati)\n      |__project_data_folder2\n                    |__ dati_mirco.mat (file dati mat file, jle file etc)\n|___ project_root(folder)\n            |__d_cod_mon_Mirco.py\n            |__some_functions.py\n            |___EEG_ANN_pipeline(folder)\n                    |__data (folder)\n                    |__helpers (folder)\n                    |__etc....\n            |___output directory (folder)\n            \n\n'

In [3]:
# need to declare:
# 1) PROJECT root directory
#  windows directories
i_dir='J:\\AI_PhD_Neuro_CNR\\Empirics\\GIT_stuff\\AI_for_all\\Contrastive_Stuff'

#  ubuntu directories
#i_dir=r'/media/zlollo/21DB-AB79/AI_PhD_Neuro_CNR/Empirics/GIT_stuff/AI_for_all/Contrastive_Stuff/'

os.chdir(i_dir)
print(os.getcwd())
from some_functions import *
# 2) NAME of folder containing data (input) directory. 
data_dir="data"
# (specific) project data folder
sub_data="Monkeys_Mirco"

# 3) PIPELINE folder name
pipe_path= "EEG-ANN-Pipeline"

# 4) OUTPUT folder: folder to store processed output
#    (if not existing is created)
out_dir="contrastive_output"




J:\AI_PhD_Neuro_CNR\Empirics\GIT_stuff\AI_for_all\Contrastive_Stuff


In [5]:
# Load path(s)
project_root, eeg_pipeline_path, default_output_dir, default_input_dir = setup_paths( data_dir,sub_data,out_dir, pipe_path,change_dir=False)


INFO:some_functions:Project root resolved to: J:\AI_PhD_Neuro_CNR\Empirics\GIT_stuff\AI_for_all\Contrastive_Stuff
INFO:some_functions:Path to 'EEG-ANN-Pipeline': J:\AI_PhD_Neuro_CNR\Empirics\GIT_stuff\AI_for_all\Contrastive_Stuff\EEG-ANN-Pipeline
INFO:some_functions:Input data path: J:\AI_PhD_Neuro_CNR\Empirics\GIT_stuff\AI_for_all\data\Monkeys_Mirco
INFO:some_functions:Output path: J:\AI_PhD_Neuro_CNR\Empirics\GIT_stuff\AI_for_all\Contrastive_Stuff\contrastive_output
INFO:some_functions:Added 'J:\AI_PhD_Neuro_CNR\Empirics\GIT_stuff\AI_for_all\Contrastive_Stuff\EEG-ANN-Pipeline' to sys.path
INFO:some_functions:Output directory ensured: J:\AI_PhD_Neuro_CNR\Empirics\GIT_stuff\AI_for_all\Contrastive_Stuff\contrastive_output


In [7]:
import sys
sys.path

['J:\\AI_PhD_Neuro_CNR\\Empirics\\GIT_stuff\\AI_for_all\\Contrastive_Stuff\\dati_Mirco',
 'C:\\Users\\loren\\Desktop\\Shared\\Società\\Outerrim\\Elaborazione DS Desktop\\dati',
 'J:\\AI_PhD_Neuro_CNR\\Empirics\\GIT_stuff\\AI_for_all\\Contrastive_Stuff\\dati_Mirco\\ C:\\Users\\loren\\models\\research\\lfads',
 'J:\\AI_PhD_Neuro_CNR\\Empirics\\GIT_stuff\\AI_for_all\\Contrastive_Stuff\\dati_Mirco\\ C:\\Users\\loren\\lfads-run-manager\\src',
 'C:\\Users\\loren\\anaconda3\\python311.zip',
 'C:\\Users\\loren\\anaconda3\\DLLs',
 'C:\\Users\\loren\\anaconda3\\Lib',
 'C:\\Users\\loren\\anaconda3',
 '',
 'C:\\Users\\loren\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\loren\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\loren\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\loren\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'J:\\AI_PhD_Neuro_CNR\\Empirics\\GIT_stuff\\AI_for_all\\Contrastive_Stuff\\EEG-ANN-Pipeline']

In [9]:
#import 
from data import LabelsDistance, TrialEEG, DatasetEEG, DatasetEEGTorch
from data.preprocessing import normalize_signals
from models import EncoderContrastiveWeights
from helpers.model_utils import plot_training_metrics, count_model_parameters, train_model
from helpers.visualization import plot_latent_trajectories_3d, plot_latents_3d
from helpers.distance_functions import *
from layers.custom_layers import _Skip, Squeeze, _Norm, _MeanAndConv
# 
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import numpy as np
import random
from torch import nn
import torch
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
import sklearn.metrics
import joblib as jl
import seaborn as sns# 
# 
from cebra import CEBRA
import cebra
from some_functions import *
# 
import matplotlib
torch.set_printoptions(sci_mode=False)

# Random Seeds
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Config GPU
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [11]:
print(sys.path)

if 'torch' in sys.modules:
    print('daje')

['J:\\AI_PhD_Neuro_CNR\\Empirics\\GIT_stuff\\AI_for_all\\Contrastive_Stuff\\dati_Mirco', 'C:\\Users\\loren\\Desktop\\Shared\\Società\\Outerrim\\Elaborazione DS Desktop\\dati', 'J:\\AI_PhD_Neuro_CNR\\Empirics\\GIT_stuff\\AI_for_all\\Contrastive_Stuff\\dati_Mirco\\ C:\\Users\\loren\\models\\research\\lfads', 'J:\\AI_PhD_Neuro_CNR\\Empirics\\GIT_stuff\\AI_for_all\\Contrastive_Stuff\\dati_Mirco\\ C:\\Users\\loren\\lfads-run-manager\\src', 'C:\\Users\\loren\\anaconda3\\python311.zip', 'C:\\Users\\loren\\anaconda3\\DLLs', 'C:\\Users\\loren\\anaconda3\\Lib', 'C:\\Users\\loren\\anaconda3', '', 'C:\\Users\\loren\\anaconda3\\Lib\\site-packages', 'C:\\Users\\loren\\anaconda3\\Lib\\site-packages\\win32', 'C:\\Users\\loren\\anaconda3\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\loren\\anaconda3\\Lib\\site-packages\\Pythonwin', 'J:\\AI_PhD_Neuro_CNR\\Empirics\\GIT_stuff\\AI_for_all\\Contrastive_Stuff\\EEG-ANN-Pipeline']
daje


In [13]:

# BUILD the model encoder. (1: cnnd1d simil cebra with skip connections)
def build_model(filters, dropout, latents, num_timepoints, chns, num_units=None, groups=1,normalize=True):
    """
    Build a cnn1d model with:
    - chns: Input channels.
    - filters: convolutional layer(s) filters.
    - latents: outpuit dimension (latent space).
    - num_timepoints: window dimension (test the optimal one).
    - dropout:  dropout.
    - num_units: optional intermediate filters.
    """
    if num_units is None:
        num_units = filters 
    layers = [
        Squeeze(),
        nn.Conv1d(chns, filters, kernel_size=2),
        nn.GELU(),
        _Skip(nn.Conv1d(filters, filters, kernel_size=3), nn.GELU()),
        _Skip(nn.Conv1d(filters, filters, kernel_size=3), nn.GELU()),
        _Skip(nn.Conv1d(filters, filters, kernel_size=3), nn.GELU()),
        nn.Conv1d(filters, latents, kernel_size=3),
    ]

    if normalize:
        layers.append(_Norm())  #

    layers.extend([
        nn.Flatten(),  # 
        #nn.Dropout(dropout),  # 
    ])

    return nn.Sequential(*layers)



#### Build the model encoder (2: cnn2d)
# 
'''
def build_model(filters, dropout, latents, num_timepoints, chns):
    
    return nn.Sequential(
        nn.Conv2d(1, filters, kernel_size=(1, num_timepoints)),
        nn.BatchNorm2d(filters),
        nn.Conv2d(filters, filters, kernel_size=(chns, 1), groups=filters),
        nn.BatchNorm2d(filters),
        nn.Dropout(dropout),
        nn.Flatten(),
        nn.Linear(filters, filters),
        nn.SELU(),
        nn.Dropout(dropout),
        nn.Linear(filters, latents)
    )

'''

'\ndef build_model(filters, dropout, latents, num_timepoints, chns):\n    \n    return nn.Sequential(\n        nn.Conv2d(1, filters, kernel_size=(1, num_timepoints)),\n        nn.BatchNorm2d(filters),\n        nn.Conv2d(filters, filters, kernel_size=(chns, 1), groups=filters),\n        nn.BatchNorm2d(filters),\n        nn.Dropout(dropout),\n        nn.Flatten(),\n        nn.Linear(filters, filters),\n        nn.SELU(),\n        nn.Dropout(dropout),\n        nn.Linear(filters, latents)\n    )\n\n'

In [91]:
###  DEFINE DISTANCES to compute the loss (In   some_functions module)
"""

- Minkowski Distance: A generalized distance metric that includes 
    both Euclidean (p=2) and Manhattan (p=1) distances as special cases. 

- Adaptive Gaussian Distance**:
- Direction Distance

- Circular Distance

"""

'\n\n- Minkowski Distance: A generalized distance metric that includes \n    both Euclidean (p=2) and Manhattan (p=1) distances as special cases. \n\n- Adaptive Gaussian Distance**:\n- Direction Distance\n\n- Circular Distance\n\n'

In [15]:
 
#### PARAMETERS ###
## sampling frequency (ms)    
fs = 1000
## Validation/test split
valid_split = 0.15
## window of each mini batch
ww = 10
# overlap
shift = 1
#
dropout = 0.5
## temperature
tau = 0.5
## network hidden layers channles
filters = 16
## output channels (latents)
latents = 3
# learning rate
l_rate = 0.0001
#
epochs = 100
#sigma_pos = 0.016
#sigma_time = 0.025
#
batch_size=1024
#
num_units = filters
#
normalize=True

In [17]:
# recall the input dir (declared upwards) and import data
input_dir = default_input_dir
# data format
d_format="mat"
# data_name
d_name='dati_mirco'
data = load_data(input_dir, d_name,d_format)
print(type(data)) # must be a dictionary

try to load: J:\AI_PhD_Neuro_CNR\Empirics\GIT_stuff\AI_for_all\data\Monkeys_Mirco\dati_mirco.mat
<class 'dict'>


In [19]:

#### define X ed Y
'''
Get the data from the imported  dictionary
Data Structure:
   - X (neural data) is a matrix T(ime)xch(annels)
   - y_... are the associated labels, which can be:
    Discrete or continuous.
    Either matrices or vectors with dimensions T×n, meaning:
        Labels exist for each time point.
        n is the number of label dimensions.


'''
X=data['m1_active_neural']
y_dir=data['m1_active_trial']
y_dir=y_dir.flatten()
y_pos=data['m1_active_pos']
#y_pos=y_pos.flatten()



In [21]:
# IDENTIFY Trials (info should  be in the data dictionary or should be derived somehow
# i.e. every change in y_dir value - direction label - 
# tipo dove y_dir cambia...with the caution that it might not change from one trial to another).

trial_id=data['m1_active_trial_id']
trial_id=trial_id.flatten()
#n_trials_=127
len_y=len(y_dir)
change_idx = np.where(np.diff(trial_id) != 0)[0] + 1
change_idx
print('ciao')

ciao


In [23]:
# the c_t vector tells the starting and endiing points of every trial
c_t=np.concatenate([[0], change_idx,[len_y]], dtype=int)
n_trials=len(c_t)-1
### check trial length (useful for graphics)
trial_len=np.diff(c_t)

# zero variance = fix (constant) length (this info will be used for plots)
if np.var(trial_len)==0:    
    constant_len=True
else:
    constant_len=False
print(constant_len)

True


In [25]:
####  Number of channels Either first (0) or second dimension (1)
chns = X.shape[1]
print(chns)

3


In [27]:
######## CREATE TRIALS adding the behavioral info we want ########
'''
 trial_ids is a vector telling us the trial at time t
 c_t tells us starting (and ending) point of each trial 
'''

trials = [
    TrialEEG(
       X[ c_t[i]:c_t[i+1]].T,  # Segnali EEG
          # Labels
          [(
             'position', y_pos[c_t[i]:c_t[i+1]].T),
           ( 'direction',y_dir[c_t[i]:c_t[i+1]].T,          
       ) ],
          # Timepoints
        np.linspace(c_t[i] / fs, c_t[i+1] / fs, c_t[i+1] - c_t[i])  
    )
    for i in range(len(c_t) - 1)
]

## check
explore_obj(trials[5])
dataset = DatasetEEG(trials)
print(dataset)
print(dataset.trials[1])
explore_obj(dataset.trials[1])
dataset.trials[0].eeg_signals[0].shape

Esplorazione dell'oggetto: TrialEEG
--------------------------------------------------
eeg_signals: array shape (3, 1500)
labels: {'position': array([[ 0.6      ,  0.6      ,  0.6      , ...,  6.5      ,  6.5      ,
         6.5      ],
       [-1.1711303, -1.1441596, -1.1250274, ..., -5.575    , -5.575    ,
        -5.575    ]], dtype=float32), 'direction': array([4., 4., 4., ..., 4., 4., 4.], dtype=float32)}
num_channels: 3
num_timepoints: 1500
plot: <bound method TrialEEG.plot of <data.eeg_dataset.TrialEEG object at 0x00000171F58F4C90>>
timepoints: array shape (1500,)
--------------------------------------------------
Labels type in __str__: multi_label
num_trials               :  41
num_channels (per trial) :  3
timepoints (per trial)   :  1500 (fix)
labels_type              :  multi_label
labels_format            :  {'position': 'ndarray', 'direction': 'ndarray'}

Numero canali = 3, numero timepoints = 1500
Istante iniziale = 1.5, tempo finale = 3.0
Labels:
  position: shape (2, 1

(1500,)

In [29]:
### Function to generate embeddings after data processing 
# (move to some_functions after generalizing for the labels)
def generate_embeddings(model, dataset_pytorch, batch_size, device):
    model.eval()
    ## 
    z, labels_position, labels_direction = [], [],[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
    with torch.no_grad():
        for i in range(0, dataset_pytorch.num_trials, batch_size):
            x = dataset_pytorch.eeg_signals[i:i+batch_size].to(device)
            l_pos = dataset_pytorch.labels['position'][i:i+batch_size,:]
            l_dir = dataset_pytorch.labels['direction'][i:i+batch_size]
            print(l_dir)
            f_x = model(x)

            z.append(f_x.cpu().numpy())
            labels_position.append(l_pos.cpu().numpy())
            labels_direction.append(l_dir.cpu().numpy().reshape(-1))

    z = np.concatenate(z)
    labels_position = np.concatenate(labels_position)
    labels_direction = np.concatenate(labels_direction)
 
    return z ,labels_direction, labels_position


In [31]:
############# FULL DATASET (No training/test/validation split) ################
'''
split dataset into windows (these are the elements going in the batches)
    ww: mini-sample dimension
    shift: window overlap (shift=1 means sampling 1-10, 2-11, 3-12...)
'''
### 
dataset_windows=dataset.create_windows(ww, shift)
print(dataset_windows)
# check
explore_obj(dataset_windows.trials[46464])

#### Convert to PyTorch datasets
# if u need to select some particular labels
#sel_lab='position' (optional argument for the next called function)

dataset_pytorch=DatasetEEGTorch(dataset_windows)
explore_obj(dataset_pytorch)

Labels type in __str__: multi_label
num_trials               :  61090
num_channels (per trial) :  3
timepoints (per trial)   :  10 (fix)
labels_type              :  multi_label
labels_format            :  {'position': 'ndarray', 'direction': 'ndarray'}

Esplorazione dell'oggetto: TrialEEG
--------------------------------------------------
eeg_signals: array shape (3, 10)
labels: {'position': array([ 0.15      , -0.28761515], dtype=float32), 'direction': array(8., dtype=float32)}
num_channels: 3
num_timepoints: 10
plot: <bound method TrialEEG.plot of <data.eeg_dataset.TrialEEG object at 0x00000171F92DD550>>
timepoints: array shape (10,)
--------------------------------------------------
{'position': 'ndarray', 'direction': 'ndarray'}
Esplorazione dell'oggetto: DatasetEEGTorch
--------------------------------------------------
_is_protocol: False
create_labels: <bound method DatasetEEGTorch.create_labels of <data.eeg_dataset_torch.DatasetEEGTorch object at 0x00000171F5900490>>
Labels typ

In [33]:
'''
# Define label distances...multi label must be a dictionary; single label can 
be  either a function or a dictionary with one element
 'direction': direction_distance
  'position':adaptive_gaussian_distance..

the following line includes both position and direction distancethat is both 
lables will be used during training
***labels_distance = LabelsDistance({'position': adaptive_gaussian_distance,'direction':direction_distance})

the following line only includes direction distance in the process
***labels_distance = LabelsDistance({'position': adaptive_gaussian_distance})

'''
#labels_distance = LabelsDistance({'direction':direction_distance})
labels_distance = LabelsDistance({'position': adaptive_gaussian_distance})

explore_obj(labels_distance)
## deliver to cuda devices (if any available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_pytorch.to_device(device)
device
batch_size=1024

dataloader_ = DataLoader(dataset_pytorch, batch_size=batch_size, shuffle=False)
## explore the dataloader
for batch in dataloader_:
    #►pippo.append(batch)
    print(type(batch)) 
    print(batch)       
    break 

Esplorazione dell'oggetto: LabelsDistance
--------------------------------------------------
get_weights: <bound method LabelsDistance.get_weights of <data.contrastive_sampling.LabelsDistance object at 0x00000171F3977210>>
label_keys: length 1
labels_distance_functions: {'position': <function adaptive_gaussian_distance at 0x00000171F58C71A0>}
multi_label: False
--------------------------------------------------
<class 'list'>
[tensor([[[[    -0.0000,     -0.0000,     -0.0000,  ...,     -0.0001,
               -0.0000,     -0.0001],
          [    -0.0000,     -0.0000,     -0.0000,  ...,     -0.0000,
               -0.0000,     -0.0000],
          [     0.0000,      0.0000,      0.0000,  ...,      0.0000,
                0.0000,      0.0000]]],


        [[[    -0.0000,     -0.0000,     -0.0000,  ...,     -0.0000,
               -0.0001,     -0.0001],
          [    -0.0000,     -0.0000,     -0.0000,  ...,     -0.0000,
               -0.0000,     -0.0000],
          [     0.0000,      0

In [35]:
#################################################################################
# Build and train the model
# update the weights according the chosen labels (same value, same weight)
# just one value for label

from data import LabelsDistance, TrialEEG, DatasetEEG, DatasetEEGTorch
from data.preprocessing import normalize_signals
from models import EncoderContrastiveWeights
filters=32
num_units=filters
epochs=300

model = EncoderContrastiveWeights(
    ### define the network (recall the parameters defined from line 209)
    layers=build_model(filters, dropout, latents, ww, chns, normalize=normalize),
    ### choose between infoNCE, supervised contrastive, weeighted contrastive loss
    #loss_type="weighted_contrastive",
    # 
    labels_distance=labels_distance,
    #labels_distance=None,
    ## If we use the labels, choose some weights
    ## if we have more label and want a different weight for every label
    ## just put a weight for each label; if we have one label or want just the 
    ## same weight for each label one value is ok
    labels_weights=[0.5],
    #labels_weights=None,
    temperature=tau,
    train_temperature=False,
    ### time contrastive loss asks either offset or window
    #positive_offset=5,  # Offset per i positivi (se usi offset)
    #positive_window=0 
    
    )

model.to(device)

print("Model architecture:", model)
print(f"Model has {count_model_parameters(model)} parameters.")
#print(f"subject is {name}")
print(f"{device}")

Model architecture: EncoderContrastiveWeights(
  (layers): Sequential(
    (0): Squeeze()
    (1): Conv1d(3, 32, kernel_size=(2,), stride=(1,))
    (2): GELU(approximate='none')
    (3): _Skip(
      (module): Sequential(
        (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
        (1): GELU(approximate='none')
      )
    )
    (4): _Skip(
      (module): Sequential(
        (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
        (1): GELU(approximate='none')
      )
    )
    (5): _Skip(
      (module): Sequential(
        (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
        (1): GELU(approximate='none')
      )
    )
    (6): Conv1d(32, 3, kernel_size=(3,), stride=(1,))
    (7): _Norm()
    (8): Flatten(start_dim=1, end_dim=-1)
  )
)
Model has 9828 parameters.
cuda


In [37]:

from data import LabelsDistance, TrialEEG, DatasetEEG, DatasetEEGTorch
from data.preprocessing import normalize_signals
from models import EncoderContrastiveWeights
filters=32
num_units=filters
epochs=300

model = EncoderContrastiveWeights(
    ### define the network (recall the parameters defined from line 209)
    layers=build_model(filters, dropout, latents, ww, chns, normalize=normalize),
    ### choose between infoNCE, supervised contrastive, weeighted contrastive loss
    #loss_type="weighted_contrastive",
    # 
    labels_distance=labels_distance,
    #labels_distance=None,
    ## If we use the labels, choose some weights
    ## if we have more label and want a different weight for every label
    ## just put a weight for each label; if we have one label or want just the 
    ## same weight for each label one value is ok
    labels_weights=[0.5],
    #labels_weights=None,
    temperature=tau,
    train_temperature=False,
    ### time contrastive loss asks either offset or window
    #positive_offset=5,  # Offset per i positivi (se usi offset)
    #positive_window=0 
    
    )

model.to(device)

print("Model architecture:", model)
print(f"Model has {count_model_parameters(model)} parameters.")
#print(f"subject is {name}")
print(f"{device}")


Model architecture: EncoderContrastiveWeights(
  (layers): Sequential(
    (0): Squeeze()
    (1): Conv1d(3, 32, kernel_size=(2,), stride=(1,))
    (2): GELU(approximate='none')
    (3): _Skip(
      (module): Sequential(
        (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
        (1): GELU(approximate='none')
      )
    )
    (4): _Skip(
      (module): Sequential(
        (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
        (1): GELU(approximate='none')
      )
    )
    (5): _Skip(
      (module): Sequential(
        (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
        (1): GELU(approximate='none')
      )
    )
    (6): Conv1d(32, 3, kernel_size=(3,), stride=(1,))
    (7): _Norm()
    (8): Flatten(start_dim=1, end_dim=-1)
  )
)
Model has 9828 parameters.
cuda


In [ ]:
# Train and evaluate the model
optimizer = torch.optim.Adam(model.parameters(), lr=l_rate)

# train on a batch (just to check if everything is ok)
batch = next(iter(dataloader_))
loss_dict = model.process_batch(batch, optimizer)

### model training
metrics = train_model(model, optimizer, dataloader_, epochs=epochs)
plot_training_metrics(metrics)

 89%|██████████████████████████████▎   | 268/300 [04:27<00:32,  1.02s/it, alignement=-4.66, uniformity=4.44, loss=-.22]

In [165]:
## GENERATE EMBEDDINGS and recover the label info 
z_0, l_dir_0, l_pos_0 = generate_embeddings(model, dataset_pytorch, batch_size, device)


tensor([3., 3., 3.,  ..., 3., 3., 3.], device='cuda:0')
tensor([3., 3., 3.,  ..., 1., 1., 1.], device='cuda:0')
tensor([1., 1., 1.,  ..., 2., 2., 2.], device='cuda:0')
tensor([2., 2., 2.,  ..., 2., 2., 2.], device='cuda:0')
tensor([2., 2., 2.,  ..., 5., 5., 5.], device='cuda:0')
tensor([5., 5., 5.,  ..., 7., 7., 7.], device='cuda:0')
tensor([7., 7., 7.,  ..., 7., 7., 7.], device='cuda:0')
tensor([7., 7., 7.,  ..., 4., 4., 4.], device='cuda:0')
tensor([4., 4., 4.,  ..., 6., 6., 6.], device='cuda:0')
tensor([6., 6., 6.,  ..., 6., 6., 6.], device='cuda:0')
tensor([6., 6., 6.,  ..., 8., 8., 8.], device='cuda:0')
tensor([8., 8., 8.,  ..., 7., 7., 7.], device='cuda:0')
tensor([7., 7., 7.,  ..., 7., 7., 7.], device='cuda:0')
tensor([7., 7., 7.,  ..., 8., 8., 8.], device='cuda:0')
tensor([8., 8., 8.,  ..., 3., 3., 3.], device='cuda:0')
tensor([3., 3., 3.,  ..., 3., 3., 3.], device='cuda:0')
tensor([3., 3., 3.,  ..., 5., 5., 5.], device='cuda:0')
tensor([5., 5., 5.,  ..., 1., 1., 1.], device='c

In [129]:
trial_len

array([1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
       1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
       1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
       1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500])

In [171]:
from some_functions import *
### generate 3d plot
trial_length=trial_len[0]
constant_len
###define the trajectories. First monkey is 1-8, second is 9-16
# first
n_traj=np.arange(8)+1
# second
#n_traj=np.arange(8)+9
#

c_s="maroon"
output_folder = default_output_dir
plot_direction_averaged_embedding(z_0, l_dir_0, n_traj,c_s,output_folder, 
                                      trial_len, constant_length=True, 
                                      ww=10)



NameError: name 'plot_direction_averaged_embedding' is not defined

In [147]:

from some_functions import *
